Feature selection

BERT integration

In [23]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install AutoModel

In [29]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


distilbert-base-uncased because:
- pre-trained BERT model
- text classification
- distilled BERT model for smaller dataset
- slightly lesser performance than bert-base-uncased
- switch to bert-base-uncased for higher accuracy and if we have more computational resources; it will be slower

maybe not BERT? Any BERT models max text length is 512 and the news texts are longer


In [31]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import torch
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
# from datasets import Dataset
from transformers import AutoTokenizer # Hugging Face Transformers
from transformers import AutoModel
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score


In [32]:
news_df = pd.read_csv('Data/raw_data/fake_or_real_news.csv') # TODO: Get cleaned data instead

label_encoder = preprocessing.LabelEncoder()
news_df['label'] = label_encoder.fit_transform(news_df['label'].tolist())

In [33]:
train_df, test_df = train_test_split(news_df, test_size=0.2)

In [34]:
import torch 
from transformers import AutoTokenizer, AutoModel, AdamW 
# Load the pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english") 
model = AutoModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english") # Example news data (replace with your actual dataset) 
inputs = tokenizer(news_df['title'].to_list(), return_tensors='pt', padding=True, truncation=True, max_length=512)
optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()  # Set the model to training mode 
for epoch in range(3): 
    optimizer.zero_grad()  # Zero out previous gradients 
    # Forward pass: Get model outputs 
    with torch.no_grad(): # Since we're just extracting embeddings, no need to compute gradients 
        outputs = model(**inputs) # Extract the CLS token embeddings (usually the first token) 
        cls_embeddings = outputs.last_hidden_state[:, 0, :].numpy()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\rkaya\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rkaya\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

c:\Users\rkaya\anaconda3\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Using logistic regression for classification

In [35]:
X_train, X_test, y_train, y_test = train_test_split(cls_embeddings, news_df['label'].to_list(), test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000) # TODO: try knn
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7056037884767167


Using KNN-3 for classification

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
    cls_embeddings, 
    news_df['label'].to_list(), 
    test_size=0.2, 
    random_state=42
)

# Initialize the K-Nearest Neighbors classifier
clf = KNeighborsClassifier(n_neighbors=3)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6511444356748224
